# Taller 1 Conceptos básicos

Autores:
- Guillermo Luigui Ubaldo Nieto Angarita
- Daniel Hernando Moyano Salamanca
- Jairo Antonio Viteri Rojas

In [140]:
!myenv\Scripts\activate


c:\Users\guill\Documents\GitHub\Intro-IA-Generativa-Taller-1-Conceptos-Basicos>() 


In [141]:
!pip install -r requirements.txt

In [142]:
import os
import requests
import nltk
import spacy
import json
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from transformers import pipeline, set_seed, DataCollatorWithPadding, BertForSequenceClassification, Trainer, TrainingArguments, BertTokenizer, BertModel, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer
from scipy.spatial.distance import cosine
import evaluate
from datasets import load_dataset
from evaluate import evaluator
from gensim.models import KeyedVectors

from tf_keras.api._v2.keras.layers import Input, Dense, SimpleRNN
from tf_keras.api._v2.keras.models import Model
from tf_keras.api._v2.keras.optimizers import SGD
from tf_keras.api._v2.keras.utils import to_categorical
from tf_keras.api._v2.keras import backend as K

from model2vec import StaticModel
from sklearn.metrics.pairwise import cosine_similarity

In [143]:
nltk.download('all')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download("omw-1.4")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[

True

In [144]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     - -------------------------------------- 0.5/12.8 MB 5.6 MB/s eta 0:00:03
     ----- ---------------------------------- 1.8/12.8 MB 5.6 MB/s eta 0:00:02
     --------- ------------------------------ 3.1/12.8 MB 6.0 MB/s eta 0:00:02
     ----------- ---------------------------- 3.7/12.8 MB 5.0 MB/s eta 0:00:02
     -------------- ------------------------- 4.7/12.8 MB 4.7 MB/s eta 0:00:02
     ------------------ --------------------- 6.0/12.8 MB 5.0 MB/s eta 0:00:02
     ---------------------- ----------------- 7.3/12.8 MB 5.1 MB/s eta 0:00:02
     --------------------------- ------------ 8.9/12.8 MB 5.4 MB/s eta 0:00:01
     -------------------------------- ------- 10.5/12.8 MB 5.7 MB/s eta 0:00:01
     -------------------------------------- - 12.3/12.8 MB 6.0 MB/s eta 0:00:01
     ---------------------------------------- 12.8/12.8 MB 5.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load 

In [145]:
# Define the URL and the output file path
url = "http://nlp.uoregon.edu/download/embeddings/glove.6B.50d.txt"
output_file = "glove.6B.50d.txt"

# Check if the file already exists
if not os.path.exists(output_file):
    print(f"Downloading {output_file}...")
    # Stream the download to handle large files
    response = requests.get(url, stream=True)
    with open(output_file, 'wb') as file:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                file.write(chunk)
    print(f"Downloaded {output_file}.")
else:
    print(f"{output_file} already exists. No download needed.")

glove.6B.50d.txt already exists. No download needed.


## 1. Introducción a NLP

### 1.1. Pregunta 1: ¿Qué es el Procesamiento de Lenguaje Natural y cuáles son sus principales aplicaciones?

El procesamiento del lenguaje natural (NLP) es un campo de la IA que permite a las máquinas comprender, interpretar, generar y manipular el lenguaje humano de forma natural, lo que facilita las interacciones entre humanos y ordenadores.

El NLP utiliza modelos estadísticos, técnicas de aprendizaje automático y, más recientemente, arquitecturas de Deep Learning como Transformers (por ejemplo, BERT y GPT). Estos modelos convierten el texto en representaciones matemáticas, como embeddings, permitiendo que las máquinas detecten patrones, relaciones semánticas y contexto.

Las aplicaciones principales del PLN incluyen:
- **Análisis de Sentimientos**: Detectar la polaridad (positiva, negativa o neutra) en textos como reseñas, redes sociales o encuestas.
- **Chatbots y Asistentes Virtuales**: Automatizar la interacción con los usuarios mediante respuestas precisas y contextuales.
- **Traducción Automática**: Transformar texto de un idioma a otro utilizando redes neuronales para mantener coherencia y fluidez.
- **Resumen Automático de Textos**: Condensar información larga en versiones más breves sin perder los puntos clave.
- **Detección de Spam y Filtrado de Contenidos**: Clasificar mensajes o contenido inapropiado para mejorar la experiencia del usuario.


### 1.1.1 Ejercicio: Investigar 6 aplicaciones actuales de NLP

- **ChatGPT (Asistente Conversacional Generativo)**: es un modelo de lenguaje basado en la arquitectura GPT-4. Se utiliza en múltiples sectores para ofrecer atención automatizada a clientes, resolver preguntas complejas y brindar soporte técnico. Su capacidad de comprender y mantener conversaciones contextuales mejora la eficiencia y reduce los tiempos de respuesta en empresas de servicios financieros, comercio electrónico y salud. Mejora la experiencia del cliente al ofrecer atención continua y personalizada, reduciendo la carga de trabajo en los centros de atención al cliente.

- **DALL-E3 (Generación de Imágenes a partir de Texto)**: Aunque se centra en la generación de imágenes, DALL-E combina PLN con modelos de visión computacional para interpretar descripciones complejas y convertirlas en representaciones visuales. Esta tecnología tiene aplicaciones en marketing, diseño y educación. Permite crear contenido visual sin necesidad de conocimientos avanzados en diseño, optimizando la producción creativa en campañas publicitarias y proyectos educativos.

- **MedPaLM (Modelo especializado en PLN para salud)**: MedPaLM es un modelo optimizado para procesar información médica, responder preguntas clínicas y brindar recomendaciones basadas en evidencia. Está diseñado para manejar textos técnicos y resolver preguntas complejas en medicina. Mejora la eficiencia de los profesionales de la salud al proporcionar resúmenes rápidos y precisos de investigaciones, diagnósticos y tratamientos, facilitando la toma de decisiones en entornos clínicos.

- **Asistentes Virtuales de Servicio al Cliente (AVSC)**: son sistemas alimentados por modelos de PLN para ofrecer soporte al cliente 24/7. Pueden responder preguntas, resolver problemas comunes y guiar a los usuarios a través de procesos complejos. Los AVSC aumentan la eficiencia operativa al reducir el tiempo de espera y atender múltiples consultas simultáneamente, permitiendo a las empresas ofrecer un mejor servicio sin aumentar la carga de trabajo en el personal. Los asistentes virtuales ayudan a reducir costos operativos y mejoran la satisfacción del cliente al proporcionar respuestas rápidas y precisas.

- **Traducción Automática Avanzada**: Servicios como Google Translate han mejorado notablemente gracias a los avances en PLN. Utilizan modelos de traducción automática neuronales que comprenden el contexto y las sutilezas del lenguaje humano. Estos sistemas facilitan la comunicación entre personas que hablan diferentes idiomas, rompiendo barreras lingüísticas en negocios, turismo y relaciones personales; ayudan a prevenir malentendidos en la comunicación y permiten el acceso a información y servicios en múltiples idiomas.

- **Análisis de Sentimientos en Redes Sociales**: herramientas tecnológicas que utilizan técnicas de PLN para evaluar las opiniones y emociones expresadas en redes sociales y reseñas en línea. Estas herramientas pueden clasificar el contenido como positivo, negativo o neutral. Permiten a las empresas y organizaciones comprender mejor la percepción pública sobre sus productos, servicios o iniciativas. Ayudan a las marcas a identificar problemas emergentes y áreas de mejora en sus ofertas, a la vez que permiten a los responsables de marketing ajustar estrategias en tiempo real basándose en la retroalimentación del consumidor.


### 1.2. Pregunta 2: Explica los conceptos de tokenización y segmentación de oraciones. ¿Por qué son importantes en NLP?

La tokenización es el proceso de dividir un texto en unidades más pequeñas llamadas tokens. Esto puede implicar descomponer el texto en oraciones, palabras, subpalabras o incluso caracteres. Es importante porque permite convertir un texto no estructurado en uno estructurado, lo que facilita su análisis estadístico o su uso en modelos de machine learning. 

#### 1.2.1 Ejercicio: Usar la biblioteca nltk o spaCy en Python para tokenizar un texto corto.

##### 1.2.1.1 Ejercicio con Nltk

##### 1.2.1.1.1 Escribir un texto

In [146]:
nltk_text = "I am learning with Nltk. It is awesome!"

##### 1.2.1.1.2 Tokenizar por oraciones

In [147]:
sentences = sent_tokenize(nltk_text)
print(f"Sentance list: {sentences}.")

Sentance list: ['I am learning with Nltk.', 'It is awesome!'].


##### 1.2.1.1.3 Tokenizar por palabras

In [148]:
words = word_tokenize(nltk_text)
print(f"Word list: {words}.")

Word list: ['I', 'am', 'learning', 'with', 'Nltk', '.', 'It', 'is', 'awesome', '!'].


##### 1.2.1.1.4 Tokenizar por subpalabras

In [149]:
stemmer = PorterStemmer()
stemmer_subwords = [stemmer.stem(word) for word in words]
print(f"Subword list: {stemmer_subwords}.")

Subword list: ['i', 'am', 'learn', 'with', 'nltk', '.', 'it', 'is', 'awesom', '!'].


In [150]:
lemmatizer = WordNetLemmatizer()
lemmatizer_subwords = [lemmatizer.lemmatize(word) for word in words]
print(f"Subword list: {lemmatizer_subwords}.")

Subword list: ['I', 'am', 'learning', 'with', 'Nltk', '.', 'It', 'is', 'awesome', '!'].


###### 1.2.1.1.5 Tokenizar por caracteres

In [57]:
characters = list(nltk_text)
print(f"Character List: {characters}")

Character List: ['I', ' ', 'a', 'm', ' ', 'l', 'e', 'a', 'r', 'n', 'i', 'n', 'g', ' ', 'w', 'i', 't', 'h', ' ', 'N', 'l', 't', 'k', '.', ' ', 'I', 't', ' ', 'i', 's', ' ', 'a', 'w', 'e', 's', 'o', 'm', 'e', '!']


#### 1.2.1.2 Ejercicio con spaCy

##### 1.2.1.2.1 Cargar el texto

In [152]:
nlp = spacy.load("en_core_web_sm")

In [153]:
spacy_text = "I am learning how to do tokenization with spacy. It is awesome!"

In [154]:
doc = nlp(spacy_text)

##### 1.2.1.2.2 Tokenizar por oraciones

In [155]:
sentences = []
for sentence in doc.sents:
    sentences.append(sentence)

sentences

[I am learning how to do tokenization with spacy., It is awesome!]

##### 1.2.1.2.2 Tokenizar por words

In [156]:
words = []
for token in doc:
    words.append(token.text)
words

['I',
 'am',
 'learning',
 'how',
 'to',
 'do',
 'tokenization',
 'with',
 'spacy',
 '.',
 'It',
 'is',
 'awesome',
 '!']

###### 1.2.1.2.3 Tokenizar por caracteres

In [157]:
characters = list(spacy_text)
characters

['I',
 ' ',
 'a',
 'm',
 ' ',
 'l',
 'e',
 'a',
 'r',
 'n',
 'i',
 'n',
 'g',
 ' ',
 'h',
 'o',
 'w',
 ' ',
 't',
 'o',
 ' ',
 'd',
 'o',
 ' ',
 't',
 'o',
 'k',
 'e',
 'n',
 'i',
 'z',
 'a',
 't',
 'i',
 'o',
 'n',
 ' ',
 'w',
 'i',
 't',
 'h',
 ' ',
 's',
 'p',
 'a',
 'c',
 'y',
 '.',
 ' ',
 'I',
 't',
 ' ',
 'i',
 's',
 ' ',
 'a',
 'w',
 'e',
 's',
 'o',
 'm',
 'e',
 '!']

## 2. Embeddings y Representaciones Vectoriales

### 2.1 Pregunta 3: ¿Qué es un embedding de palabras y cómo representa el significado de una palabra en el espacio vectorial?

Las incrustaciones de palabras (Embeddings) son representaciones vectoriales densas de palabras que capturan sus significados, propiedades sintácticas y relaciones con otras palabras. las incrustaciones de palabras mapean palabras en un espacio vectorial continuo donde las palabras semánticamente similares se encuentran cerca unas de otras.
Un embedding es simplemente un vector de longitud fija, por ejemplo, de 100 o 300 dimensiones. Cada palabra se representa como un punto en este espacio de alta dimensionalidad, y las relaciones entre las palabras se pueden medir usando distancias o ángulos entre los vectores.


#### 2.1.1 Ejercicio: Cargar un modelo de word embeddings de Hugging Face (por ejemplo, glove o word2vec) y visualizar los embeddings de palabras similares

In [158]:
# Load a pretrained Model2Vec model
model = StaticModel.from_pretrained("minishlab/M2V_base_glove")

In [159]:
# Define the words you want to compare
word1 = "table"
word2 = "chair"

In [160]:
# Compute embeddings for both words
embeddings = model.encode([word1, word2])

In [161]:
# Calculate cosine similarity
similarity = cosine_similarity([embeddings[0]], [embeddings[1]])
print(f"Cosine similarity between '{word1}' and '{word2}': {similarity[0][0]}")

Cosine similarity between 'table' and 'chair': 0.26632168889045715


### 2.2 Pregunta 4: Explica las diferencias entre embeddings estáticos y dinámicos.

**Los embeddings estáticos**, como GloVe o Word2Vec, generan una representación vectorial fija para cada palabra en el vocabulario. Estas representaciones se calculan una vez y no cambian según el contexto en el que se encuentra la palabra y tienen las siguientes características:

- **Contexto fijo**: Cada palabra tiene una única representación (por ejemplo, "banco" como entidad financiera y "banco" como lugar para sentarse tienen la misma representación).

- **Rápido**: Son computacionalmente más eficientes y requieren menos recursos de memoria.

- **Limitaciones**: No capturan el significado contextual que puede variar en diferentes oraciones.

Por otra parte, los **embeddings dinámicos** como BERT (Bidirectional Encoder Representations from Transformers), generan representaciones que dependen del contexto en que se usa la palabra. La ubicación de la palabra dentro de una frase influye en su vector de representación y tienen las siguientes características:

- **Contexto variable**: Cada palabra puede tener múltiples representaciones dependiendo de su contexto; por ejemplo, "banco" en "fui al banco" vs. "me senté en el banco".

- **Más preciso**: Capturan significados más matizados y complejos, lo que resulta en una mejor comprensión semántica.
- **Limitaciones**: mayor carga computacional porque requieren más recursos para calcular y almacenar, y suelen ser más lentos debido a su complejidad.


#### 2.2.1 Ejercicio: Comparar GloVe (estático) con BERT (dinámico) mediante un ejemplo práctico en Python.

#### 2.2.1.1 Definir frases que se van a utilizar para la comparación

In [162]:
sentences = [
    "The bank is near the river.",
    "I need to open a bank account."
]

#### 2.2.1.2 Cargar modelo BERT para embeddings

In [163]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

c:\Users\guill\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


#### 2.2.1.3 Cargar GloVe

In [164]:

glove_path = "glove.6B.50d.txt"  # Actualiza esta ruta a tu archivo de GloVe
glove_model = KeyedVectors.load_word2vec_format(glove_path, binary=False, no_header=True)

#### 2.2.1.4 Función para obtener la representación de BERT de la palabra "bank"

In [165]:
def get_bert_embedding(word, sentences):
    # Tokenizamos las frases
    inputs = tokenizer(sentences, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    # Extraer el índice de la palabra "bank" y calcular su embedding
    embeddings = outputs.last_hidden_state
    bank_embeddings = []
    
    for i, sentence in enumerate(sentences):
        token_ids = inputs['input_ids'][i]
        # Obtener el índice del token "bank"
        bank_token_index = torch.where(token_ids == tokenizer.encode(word, add_special_tokens=False)[0])[0]
        if len(bank_token_index) > 0:
            # Usar el primer token "bank" en caso de que haya subpalabras
            bank_embedding = embeddings[i][bank_token_index[0]]
            bank_embeddings.append(bank_embedding)
    
    return bank_embeddings

#### 2.2.1.5 Función para obtener la representación de GloVe de la palabra "bank"

In [166]:
def get_glove_embedding(word):
    return glove_model[word] if word in glove_model else np.zeros((50,))

#### 2.2.1.6 Obtener embeddings de "bank" usando BERT

In [167]:
bank_embeddings_bert = get_bert_embedding("bank", sentences)

#### 2.2.1.7 Obtener el embedding de "bank" usando GloVe

In [168]:
bank_embedding_glove = get_glove_embedding("bank")

#### 2.2.1.8 Resultados

In [169]:
print("Embedding de 'bank' usando BERT (para 'The bank is near the river.'):")
print(bank_embeddings_bert[0].numpy())  # Embedding de BERT para el primer contexto

Embedding de 'bank' usando BERT (para 'The bank is near the river.'):
[-4.15259391e-01 -4.03172016e-01 -1.56830207e-01 -1.48958042e-01
  6.45179331e-01  1.56543210e-01 -2.39042610e-01  5.84185302e-01
  1.75475270e-01 -3.09580535e-01  5.02860487e-01  4.69578952e-02
  2.99013890e-02  1.61932245e-01 -1.04944110e+00  1.39990002e-01
  2.48269081e-01  9.99445692e-02  7.20661461e-01 -7.00905100e-02
 -1.06558166e-02  2.31066838e-01  5.12047336e-02 -3.61906700e-02
  1.88942119e-01  1.89230755e-01  1.26893699e-01  5.60914397e-01
  1.61382228e-01  9.01738629e-02  8.60862851e-01  3.07804607e-02
  4.03439730e-01  2.39335448e-01 -3.18555176e-01 -2.12430239e-01
 -2.50645727e-02 -8.41783807e-02 -4.26190972e-01 -7.48774260e-02
  3.24836910e-01 -3.09946060e-01 -3.60662490e-01  4.18322623e-01
 -2.27934837e-01 -3.88941944e-01 -2.87400961e-01  1.61438018e-01
 -1.42638907e-01  3.23887169e-01 -4.21454281e-01  3.96718830e-01
 -3.54039967e-01  4.15670872e-03  2.78093576e-01  4.79878694e-01
 -2.82103240e-01 -2.

In [170]:
print("\nEmbedding de 'bank' usando BERT (para 'I need to open a bank account.'):")
print(bank_embeddings_bert[1].numpy())  # Embedding de BERT para el segundo contexto


Embedding de 'bank' usando BERT (para 'I need to open a bank account.'):
[ 1.34544182e+00 -3.64325285e-01  2.19070897e-01 -4.78106588e-01
  1.16727471e+00  2.66221792e-01 -5.94738126e-01  9.01681304e-01
  1.88391611e-01 -1.82483792e-01 -3.08255013e-02 -5.01000643e-01
 -2.47993823e-02  3.79295088e-02 -1.83687508e-01 -6.09300256e-01
  5.37129462e-01  1.10333264e+00  9.77232575e-01  2.27195576e-01
 -1.17837656e+00  3.11654568e-01  3.37865591e-01  4.11539912e-01
  5.41460216e-01  7.23480582e-01  1.98293418e-01 -8.26220512e-02
 -3.97272147e-02 -1.31385767e+00  1.50691271e+00  1.09790847e-01
  1.17916264e-01  7.60836363e-01  6.30491257e-01  6.13985360e-02
 -6.35443926e-01  2.56587148e-01 -1.16643834e+00 -1.99906275e-01
  3.05586699e-02 -8.61756027e-01 -1.06976196e-01  2.49178708e-01
 -6.13669693e-01 -1.85206801e-01 -1.54895768e-01  8.44709456e-01
  1.59066319e-02 -1.23520538e-01  1.87116712e-01  7.65666723e-01
 -5.32485485e-01 -8.40332866e-01  3.84237051e-01  2.18302414e-01
 -7.17905462e-01

In [171]:
print("\nEmbedding de 'bank' usando GloVe:")
print(bank_embedding_glove)  # Embedding de GloVe


Embedding de 'bank' usando GloVe:
[ 0.66488  -0.11391   0.67844   0.17951   0.6828   -0.47787  -0.30761
  0.17489  -0.70512  -0.55022   0.1514    0.10214  -0.45063  -0.33069
  0.056133  1.2271    0.55607  -0.68297   0.037364  0.70266   1.9093
 -0.61483  -0.83329  -0.3023   -1.1118   -1.55      0.2604    0.22957
 -1.0375   -0.31789   3.5091   -0.25871   1.0151    0.65927  -0.18231
 -0.75859  -0.30927  -0.91678   1.0633   -0.66761  -0.37464  -0.29143
  0.65606  -0.44642  -0.075495 -1.0552   -0.60501   0.73582   1.0139
 -0.27749 ]


Embeddings de BERT:

- Los vectores de "bank" obtenidos de BERT para ambas frases son distintos, lo que indica que el modelo ajusta la representación según el contexto.

- Al imprimir los embeddings de "bank" en cada contexto, podrás observar cómo varían los valores.

Embedding de GloVe:

- El embedding de "bank" de GloVe es único y no cambia, ya que no toma en cuenta el contexto. Este vector será el mismo independientemente de en qué frase se utilice.

## 3. Modelos Basados en Redes Neuronales

### 3.1 Pregunta 5: ¿Qué son las redes neuronales recurrentes (RNN) y las LSTM, y cómo se usan en NLP?

Las **Redes Neuronales Recurrentes (RNN)** son un tipo de arquitectura de red neuronal diseñada para procesar datos secuenciales. A diferencia de las redes neuronales tradicionales, donde las entradas y salidas son independientes, las RNN mantienen una memoria interna que les permite recordar información de entradas anteriores. Esto es útil en tareas como la traducción automática, el reconocimiento de voz y el análisis de texto, donde el contexto es crucial para la predicción.

Uno de los principales problemas que enfrentan las RNN es el **desvanecimiento del gradiente**, que dificulta el aprendizaje de dependencias a largo plazo. A medida que se retropropagan los errores a través del tiempo, los gradientes pueden volverse extremadamente pequeños, lo que impide que la red aprenda correctamente.

En ese sentido, las **LSTM** (Long Short-Term Memory ó Memoria a Largo Corto Plazo, en español) son una variante avanzada de las RNN que fueron diseñada para aprender y recordar dependencias a largo plazo en secuencias de datos, abordando el problema del desvanecimiento del gradiente. Las LSTM incorporan una estructura más compleja con celdas de memoria y tres puertas:
- **Puerta de entrada**: controla la cantidad de información nueva que se almacena.
- **Puerta de olvido**: decide qué información debe ser descartada.
- **Puerta de salida**: regula qué información se utiliza para generar la salida.

Esta arquitectura permite a las LSTM recordar información durante períodos más largos, lo que es crucial en tareas donde el contexto puede estar lejos en la secuencia.

 

**Aplicaciones en Procesamiento del Lenguaje Natural (PNL)**

Las RNN y LSTM son ampliamente utilizadas en NLP para tareas como:
- **Traducción automática**: donde el modelo debe entender contextos complejos.
- **Análisis de sentimientos**: donde se requiere comprender la secuencia completa para hacer una predicción precisa.
- **Generación de texto**: donde el modelo genera texto coherente basado en entradas previas.


#### 3.1.1 Ejercicio: Implementar una RNN simple para predicción de secuencias en Python.

[![Introducción a las Redes Neuronales Recurrentes](https://markdown-videos-api.jorgenkh.no/url?url=https%3A%2F%2Fyoutu.be%2FbKkjQx_PS_M)](https://youtu.be/bKkjQx_PS_M)

[![GENERACIÓN DE TEXTO con Redes Recurrentes en Python (Tutorial)](https://markdown-videos-api.jorgenkh.no/url?url=https%3A%2F%2Fwww.youtube.com%2Fwatch%3Fv%3DaA9QaPu_QpA)](https://www.youtube.com/watch?v=aA9QaPu_QpA)

Tomado de: https://www.youtube.com/watch?v=aA9QaPu_QpA

##### 3.1.1.1 Lectura de datos

In [172]:
nombres = open('nombres_dinosaurios.txt','r').read()
nombres = nombres.lower()

In [173]:
alfabeto = list(set(nombres))
tam_datos, tam_alfabeto = len(nombres), len(alfabeto)

In [174]:
car_a_ind = { car:ind for ind,car in enumerate(sorted(alfabeto))}

In [175]:
ind_a_car = { ind:car for ind,car in enumerate(sorted(alfabeto))}

##### 3.1.1.2 Modelo RNN

In [176]:
n_a = 25    # Número de unidades en la capa oculta
entrada  = Input(shape=(None,tam_alfabeto))
a0 = Input(shape=(n_a,))

In [177]:
celda_recurrente = SimpleRNN(n_a, activation='tanh', return_state = True)

In [178]:
capa_salida = Dense(tam_alfabeto, activation='softmax')

In [179]:
hs, _ = celda_recurrente(entrada, initial_state=a0)
salida = []
salida.append(capa_salida(hs))

In [180]:
modelo = Model([entrada,a0],salida)

In [181]:
opt = SGD(learning_rate=0.0005)
modelo.compile(optimizer=opt, loss='categorical_crossentropy')

##### 3.1.1.3 Ejemplos de Entrenamiento

In [182]:
with open("nombres_dinosaurios.txt") as f:
    ejemplos = f.readlines()
ejemplos = [x.lower().strip() for x in ejemplos]
np.random.shuffle(ejemplos)

In [183]:
def train_generator():
    while True:
        # Tomar un ejemplo aleatorio
        ejemplo = ejemplos[np.random.randint(0,len(ejemplos))]

        # Convertir el ejemplo a representación numérica
        X = [None] + [car_a_ind[c] for c in ejemplo]

        # Crear "Y", resultado de desplazar "X" un caracter a la derecha
        Y = X[1:] + [car_a_ind['\n']]

        # Representar "X" y "Y" en formato one-hot
        x = np.zeros((len(X),1,tam_alfabeto))
        onehot = to_categorical(X[1:],tam_alfabeto).reshape(len(X)-1,1,tam_alfabeto)
        x[1:,:,:] = onehot
        y = to_categorical(Y,tam_alfabeto).reshape(len(X),tam_alfabeto)

        # Activación inicial (matriz de ceros)
        a = np.zeros((len(X), n_a))

        yield [x, a], y

##### 3.1.1.4 ENTRENAMIENTO

In [185]:
BATCH_SIZE = 80			# Número de ejemplos de entrenamiento a usar en cada iteración
NITS = 1000			# Número de iteraciones

for j in range(NITS):
    historia = modelo.fit_generator(train_generator(), steps_per_epoch=BATCH_SIZE, epochs=1, verbose=0)

    # Imprimir evolución del entrenamiento cada 1000 iteraciones
    if j%100 == 0:
        print('\nIteración: %d, Error: %f' % (j, historia.history['loss'][0]) + '\n')


C:\Users\guill\AppData\Local\Temp\ipykernel_182108\1057266624.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  historia = modelo.fit_generator(train_generator(), steps_per_epoch=BATCH_SIZE, epochs=1, verbose=0)



Iteración: 0, Error: 3.186241


Iteración: 100, Error: 2.937734


Iteración: 200, Error: 2.768003


Iteración: 300, Error: 2.671154


Iteración: 400, Error: 2.576772


Iteración: 500, Error: 2.566090


Iteración: 600, Error: 2.501309


Iteración: 700, Error: 2.463476


Iteración: 800, Error: 2.492368


Iteración: 900, Error: 2.394192



##### 3.1.1.5 Generación de nombres

In [186]:
def generar_nombre(modelo,car_a_num,tam_alfabeto,n_a):
    # Inicializar x y a con ceros
    x = np.zeros((1,1,tam_alfabeto,))
    a = np.zeros((1, n_a))

    # Nombre generado y caracter de fin de linea
    nombre_generado = ''
    fin_linea = '\n'
    car = -1

    # Iterar sobre el modelo y generar predicción hasta tanto no se alcance
    # "fin_linea" o el nombre generado llegue a los 50 caracteres
    contador = 0
    while (car != fin_linea and contador != 50):
          # Generar predicción usando la celda RNN
          a, _ = celda_recurrente(K.constant(x), initial_state=K.constant(a))
          y = capa_salida(a)
          prediccion = K.eval(y)

          # Escoger aleatoriamente un elemento de la predicción (el elemento con
          # con probabilidad más alta tendrá más opciones de ser seleccionado)
          ix = np.random.choice(list(range(tam_alfabeto)),p=prediccion.ravel())

          # Convertir el elemento seleccionado a caracter y añadirlo al nombre generado
          car = ind_a_car[ix]
          nombre_generado += car

          # Crear x_(t+1) = y_t, y a_t = a_(t-1)
          x = to_categorical(ix,tam_alfabeto).reshape(1,1,tam_alfabeto)
          a = K.eval(a)

          # Actualizar contador y continuar
          contador += 1

          # Agregar fin de línea al nombre generado en caso de tener más de 50 caracteres
          if (contador == 50):
            nombre_generado += '\n'

    return nombre_generado

In [187]:
# Generar 100 ejemplos de nombres generados por el modelo ya entrenado
lista_de_nombres = []
for i in range(100):
    lista_de_nombres.append(generar_nombre(modelo,car_a_ind,tam_alfabeto,n_a))

In [188]:
lista_de_nombres = sorted(lista_de_nombres, key=len)

In [189]:
lista_de_nombres

['\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 's\n',
 'h\n',
 's\n',
 's\n',
 'n\n',
 's\n',
 'ik\n',
 'os\n',
 'tos\n',
 'oxl\n',
 'rbo\n',
 'aos\n',
 'tns\n',
 'cos\n',
 'nkr\n',
 'oun\n',
 'woir\n',
 'haon\n',
 'usro\n',
 'nnys\n',
 'yonos\n',
 'usahs\n',
 'mrpxn\n',
 'nskts\n',
 'matos\n',
 'torus\n',
 'algws\n',
 'hiiis\n',
 'gporus\n',
 'laurus\n',
 'sureil\n',
 'scaoeu\n',
 'onoros\n',
 'nsanji\n',
 'gauistn\n',
 'aagrcot\n',
 'suroqzt\n',
 'hucotts\n',
 'osthsahg\n',
 'olernpos\n',
 'inicoeus\n',
 'irurilus\n',
 'uhirzotu\n',
 'ocnosrlur\n',
 'aulmgenus\n',
 'luferlros\n',
 'anisesata\n',
 'fnrmelura\n',
 'ectodnnps\n',
 'silykptaus\n',
 'weruchruss\n',
 'jsuesaicus\n',
 'nojasciuqoj\n',
 'atlhrurorus\n',
 'esdotloyntj\n',
 'glyvobanues\n',
 'dutosnanous\n',
 'rurticrurur\n',
 'iromitnhsfpn\n',
 'gnrroentmaos\n',
 'apauclrfolcs\n',
 'oktseulaloom\n',
 'dotsippepnits\n',
 'asttsipcgontl\n',
 'orysargryasau\n',
 'hurtiammepucr\n',
 'oophgsaeahopcn\n',
 'yrdiyerenualor\n',
 'o

### 3.2 Pregunta 6: ¿Qué son los transformers y por qué han reemplazado en gran medida a las RNN en NLP?

Un transformador es un modelo de una rede neural que aprende del contexto de una secuencia de datos para generar nuevos datos. Su funcionamiento empieza con la tokenización en donde descompone el texto a unidades llamadas tokens. Estos tokens se convierten en valores numéricos mediante los embeddings y se les asigna una posición de cada palabra con la codificación posicional. La información se envía al codificador en donde ejecuta un mecanismo de autoatención y un mecanismo de mecanismo de atención multicabezal con el fin de devolver la representación de la entrada. Por último, envía el texto al decodificador para generar el texto objetivo. Esta arquitectura ha reemplazado a las redes neuronales recurrentes (RNNs) porque procesan el texto de forma paralela en cambio de forma secuencial lo cual mejorar el rendimiento del tiempo, pueden seguir el contexto del tema aún si la secuencia del texto es muy largo y aprenden más rápido.

#### 3.2.1 Ejercicio: Utilizar el modelo de transformers BERT o GPT-2 de Hugging Face para completar una frase simple.

In [190]:
text = "Being a computer scientist is all about"

##### 3.2.1.1 Utilizar EleutherAI

In [191]:
eleutherAI_generator = pipeline("text-generation", model="EleutherAI/gpt-neo-125m")

In [192]:
set_seed(85)
eleutherAI_prediction = eleutherAI_generator(text)
eleutherAI_prediction

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
c:\Users\guill\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'Being a computer scientist is all about the ability to analyze and understand the world. The world is a'}]

##### 3.2.1.2 Utilizar GPT 2

In [193]:
set_seed(42)
gpt2_generator = pipeline("text-generation", model="gpt2") 

In [68]:
gpt2_prediction = gpt2_generator(text)
gpt2_prediction

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Being a computer scientist is all about getting what you want. But you're still a little frustrated when you can't make progress, and those things are what drive your problems: your frustration with your work, your frustration with technology.\n\nThere is"}]

## 4. Modelos Pre-entrenados y Fine-Tuning

### 4.1 Pregunta 7: ¿Qué significa que un modelo esté preentrenado? ¿Cómo se puede ajustar para una tarea específica?

**Modelo preentrenado**
Un modelo de lenguaje es un tipo de algoritmo de machine learning diseñado para predecir la siguiente palabra en una oración, basándose en los segmentos predecesores. Está basado la arquitectura de Transformers. Los modelos de lenguaje pre-entrenados se entrenan con grandes cantidades de datos de texto, que permite a los LLM comprender los principios fundamentales que rige el idioma en el uso de las palabras y su disposición en el lenguaje natural.

**Fine-Tuning**
El Fine-tuning es el proceso de tomar un modelo preentrenado y entrenarlo mas allá de lo preestablecido en un conjunto de datos mas especifico en el dominio. Esto ofrece ventajas como menor gasto de maquina en el cálculo y mejora la capacidad de procesamiento de los datos. Con ello mejora el desempeño en tareas específicas, como análisis de sentimientos, question answering, resumen de documentos, con mayor precisión. 



#### 4.1.1 Realizar el fine-tuning de un modelo pre-entrenado de Hugging Face (BERT o RoBERTa) para clasificación de texto.

In [194]:
dataset = load_dataset('glue', 'sst2')

In [195]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [196]:
def preprocess_function(examples):
    return tokenizer(examples['sentence'], truncation=True, padding=True, max_length=128)

tokenized_dataset = dataset.map(preprocess_function, batched=True)

In [198]:
# Define the sampling proportion, e.g., 10%
train_proportion = 0.01
test_proportion = 0.01

# Calculate sample sizes based on the proportions
train_sample_size = int(len(tokenized_dataset['train']) * train_proportion)
test_sample_size = int(len(tokenized_dataset['validation']) * test_proportion)

# Randomly select samples from the tokenized data
train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(train_sample_size))
test_dataset = tokenized_dataset["validation"].shuffle(seed=42).select(range(test_sample_size))

In [199]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

c:\Users\guill\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [200]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [201]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
)

trainer.train()

 33%|███▎      | 43/129 [01:37<02:31,  1.76s/it]

{'eval_loss': 0.4728577435016632, 'eval_runtime': 0.2895, 'eval_samples_per_second': 27.636, 'eval_steps_per_second': 3.455, 'epoch': 1.0}


 67%|██████▋   | 86/129 [03:13<01:15,  1.76s/it]

{'eval_loss': 0.10971170663833618, 'eval_runtime': 0.2852, 'eval_samples_per_second': 28.051, 'eval_steps_per_second': 3.506, 'epoch': 2.0}


100%|██████████| 129/129 [04:50<00:00,  2.26s/it]

{'eval_loss': 0.08462903648614883, 'eval_runtime': 0.3078, 'eval_samples_per_second': 25.994, 'eval_steps_per_second': 3.249, 'epoch': 3.0}
{'train_runtime': 290.9157, 'train_samples_per_second': 6.94, 'train_steps_per_second': 0.443, 'train_loss': 0.38769806263058687, 'epoch': 3.0}


TrainOutput(global_step=129, training_loss=0.38769806263058687, metrics={'train_runtime': 290.9157, 'train_samples_per_second': 6.94, 'train_steps_per_second': 0.443, 'total_flos': 66670388650860.0, 'train_loss': 0.38769806263058687, 'epoch': 3.0})

In [202]:
results = trainer.evaluate()
print(results)

100%|██████████| 1/1 [00:00<00:00, 109.04it/s]

{'eval_loss': 0.08462903648614883, 'eval_runtime': 0.3151, 'eval_samples_per_second': 25.386, 'eval_steps_per_second': 3.173, 'epoch': 3.0}


### 4.2 **Pregunta 8**: ¿Qué es el enmascaramiento de palabras en el preentrenamiento de modelos como BERT?

El **enmascaramiento de palabras** es una técnica fundamental en el preentrenamiento de modelos de lenguaje como BERT. Consiste en enmascarar aleatoriamente algunas palabras en una oración y entrenar al modelo para predecir la palabra original en función del contexto proporcionado por las palabras circundantes. Es importante porque permite:
- **Comprensión profunda del lenguaje**: Al intentar predecir palabras faltantes, el modelo aprende a captar las relaciones semánticas y sintácticas entre las palabras, lo que le permite desarrollar una comprensión profunda del lenguaje.
- **Generación de contexto**: El enmascaramiento fuerza al modelo a generar un contexto rico y coherente para cada palabra enmascarada, lo que es esencial para tareas posteriores como la generación de texto y la traducción.

**Ejemplo**: Si tenemos la frase "El gato <mask> un ratón", el modelo podría ser entrenado para predecir que la palabra faltante es "cazó".


#### 4.1.2 Ejercicio: Usar BERT para predecir palabras enmascaradas en una oración

In [203]:
text = "Being a computer scientist is all about [MASK]."

In [204]:
bert_mask = pipeline("fill-mask", model="bert-base-uncased") 

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
c:\Users\guill\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default.

In [205]:
bert_predition = bert_mask(text)
bert_predition

[{'score': 0.11361545324325562,
  'token': 7588,
  'token_str': 'computers',
  'sequence': 'being a computer scientist is all about computers.'},
 {'score': 0.06099292263388634,
  'token': 2671,
  'token_str': 'science',
  'sequence': 'being a computer scientist is all about science.'},
 {'score': 0.05559048056602478,
  'token': 2009,
  'token_str': 'it',
  'sequence': 'being a computer scientist is all about it.'},
 {'score': 0.048937730491161346,
  'token': 2470,
  'token_str': 'research',
  'sequence': 'being a computer scientist is all about research.'},
 {'score': 0.03174487128853798,
  'token': 8785,
  'token_str': 'math',
  'sequence': 'being a computer scientist is all about math.'}]

## 5. Tareas Clásicas de NLP

### 5.1 **Pregunta 9**: ¿Qué es la clasificación de texto y qué modelos se suelen utilizar para esta tarea?

Es una tarea del NLP (procesamiento del lenguaje natural) que consiste en asignar una o más categorías a un fragmento de texto. Este modelo tiene como objeto el aprendizaje automatizado de textos en base a su contenido. Se puede utilizar como análisis de sentimientos, detección de spam, categorización de noticias, clasificación de temas o clusterización, etc. 

#### **Modelos**

•   **Regresión logística:** Son modelos lineales clásicos que se utilizan para tareas simples de clasificación de texto.

•   **Basados en redes neuronales:** Procesa secuencias de texto de longitud variable, lo que les permite modelar mejor la estructura y el contexto de las palabras en una oración.

•   **Word embeddings:** Son técnicas que convierten palabras en vectores en un espacio continuo, donde las palabras con significados similares están más cerca entre sí. 

•   **BERT:** Es un modelo preentrenado que aprende relaciones bidireccionales entre palabras. Para clasificación de texto, se ajusta para predecir la etiqueta de una secuencia de texto. 

•   **DistilBERT:** Es una versión más pequeña y eficiente de BERT que mantiene gran parte del rendimiento, pero con menos parámetros, lo que lo hace más rápido de entrenar y ejecutar.

•   **GPT (Generative Pretrained Transformer):** Aunque GPT está diseñado para tareas de generación de texto, también se puede ajustar para tareas de clasificación.


#### 5.1.1 **Ejercicio**: Usar un modelo de Hugging Face (distilBERT) para clasificar un conjunto de datos de sentimientos (usar el dataset de Rotten tomatoes)

In [206]:
dataset_rotten_tomatos = load_dataset("rotten_tomatoes", split="train").shuffle(seed=42).select(range(10))

In [207]:
model_distilbert = pipeline("text-classification", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")

In [208]:
results = model_distilbert(dataset_rotten_tomatos['text'], truncation=True, max_length=512)

In [209]:
# Convertir resultados a DataFrame y mostrar
results_rotten_tomatos_df = pd.DataFrame(results)
results_rotten_tomatos_df['text'] = dataset_rotten_tomatos['text']
results_rotten_tomatos_df = results_rotten_tomatos_df[['text', 'label', 'score']]
print(results_rotten_tomatos_df)

                                                text     label     score
0  . . . plays like somebody spliced random momen...  NEGATIVE  0.999571
1  michael moore has perfected the art of highly ...  POSITIVE  0.999870
2  . . . too gory to be a comedy and too silly to...  NEGATIVE  0.998669
3  a graceful , contemplative film that gradually...  POSITIVE  0.999831
4  the fact that the 'best part' of the movie com...  NEGATIVE  0.998292
5  the thriller side of this movie is falling fla...  NEGATIVE  0.999801
6                          more maudlin than sharp .  NEGATIVE  0.999730
7  shot like a postcard and overacted with all th...  NEGATIVE  0.999769
8  oedekerk mugs mercilessly , and the genuinely ...  NEGATIVE  0.999582
9  haneke keeps us at arm's length . guided more ...  NEGATIVE  0.984584


### 5.2 **Pregunta 10**: ¿Qué es el análisis de sentimientos y cómo se relaciona con la clasificación de texto?

El **análisis de sentimientos** es una técnica del procesamiento de lenguaje natural (PNL) que se utiliza para identificar y clasificar las opiniones expresadas en un texto. Generalmente, estas opiniones se categorizan en tres clases: positivas, negativas o neutras. Este proceso implica el uso de técnicas de minería de textos y aprendizaje automático para extraer información subjetiva, permitiendo a las organizaciones comprender mejor las emociones y actitudes de los usuarios hacia productos, servicios o temas específicos. El análisis de sentimientos se considera una forma específica de clasificación de texto, donde el objetivo es asignar una etiqueta (polaridad) a un fragmento de texto basado en su contenido emocional. Esto puede incluir:
- **Clasificación binaria**: donde el texto se clasifica como positivo o negativo.
- **Clasificación multiclase**: que puede incluir categorías adicionales como neutral o mixta.

El análisis de sentimientos puede ser visto como un subtipo dentro del campo más amplio de la clasificación de texto, que abarca tareas como la categorización automática de documentos, el etiquetado temático y la detección de spam.

#### 5.2.1 Ejercicio: Utilizar Hugging Face para hacer análisis de sentimientos en un conjunto de datos de reseñas (usar el dataset de IMDB).

##### 5.2.1.1 Cargar 10 datos del dataset IMDB 

In [210]:
dataset = load_dataset("imdb", split="train").shuffle(seed=42).select(range(10))

##### 5.2.1.2 Inicializar el pipeline de análisis de sentimientos

In [211]:
sentiment_pipeline = pipeline("sentiment-analysis", "lxyuan/distilbert-base-multilingual-cased-sentiments-student")

##### 5.2.1.3 Analizar los sentimientos de los textos del dataset de IMDB

In [212]:
results = sentiment_pipeline(dataset['text'], truncation=True, max_length=512)

##### 5.2.1.4 Imprimir los resultados

In [213]:

# Convertir resultados a DataFrame y mostrar
results_df = pd.DataFrame(results)
results_df['text'] = dataset['text']
results_df = results_df[['text', 'label', 'score']]
print(results_df)

                                                text     label     score
0  There is no relation at all between Fortier an...  positive  0.466838
1  This movie is a great. The plot is very true t...  positive  0.963030
2  George P. Cosmatos' "Rambo: First Blood Part I...  negative  0.684366
3  In the process of trying to establish the audi...  negative  0.714222
4  Yeh, I know -- you're quivering with excitemen...  negative  0.608305
5  While this movie's style isn't as understated ...  positive  0.646290
6  I give this movie 7 out of 10 because the vill...  positive  0.418127
7  really awful... lead actor did OK... the film,...  negative  0.524330
8  Good grief I can't even begin to describe how ...  negative  0.838564
9  Home Room deals with a Columbine-like high-sch...  negative  0.594892


### 5.3 **Pregunta 11**: Explica el concepto de etiquetado de secuencias (named entity recognition, NER). ¿Para qué se utiliza?

El **etiquetado de secuencias** o NER es una tarea de NLP que consiste en identificar y clasificar entidades nombradas en un texto. Las entidades nombradas pueden ser personas, organizaciones, ubicaciones, fechas, etc. Se utiliza para:
- **Extracción de información**: NER es fundamental para extraer información relevante de grandes volúmenes de texto, como noticias, artículos científicos y documentos legales.
- **Análisis de sentimientos**: Al identificar las entidades, se pueden analizar los sentimientos asociados a ellas.
- **Sistemas de recomendación**: NER puede utilizarse para personalizar recomendaciones basadas en las preferencias del usuario.
- **Chatbots y asistentes virtuales**: Los Chatbots pueden utilizar NER para comprender mejor las consultas de los usuarios y proporcionar respuestas más precisas.

**Ejemplo**: En la frase "El presidente de Estados Unidos visitó París el 14 de julio", NER identificaría "Estados Unidos" como una ubicación, "París" como otra ubicación y "14 de julio" como una fecha.

#### 5.3.1 **Ejercicio**: Implementar un modelo NER utilizando spaCy o Hugging Face y aplicar etiquetas a un conjunto de texto.

In [130]:
text = """"
Trinamool Congress leader Mahua Moitra has moved the Supreme Court against her expulsion from the Lok Sabha over the cash-for-query allegations against her. Moitra was ousted from the Parliament last week after the Ethics Committee of the Lok Sabha found her guilty of jeopardising national security by sharing her parliamentary portal's login credentials with businessman Darshan Hiranandani."""

##### 5.3.1.1 Utilizar spaCy

In [214]:
import spacy

In [215]:
nlp = spacy.load("en_core_web_sm")

In [216]:
doc = nlp(text)

In [217]:
for word in doc.ents:
    print(f"This word {word.text} is this {word.label_} entity.")

#### 3.1.1.2 Utilizar Hugging Face

In [218]:
bert_ner = pipeline("ner", model="dslim/bert-base-NER")

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
c:\Users\guill\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check t

In [219]:
ner_results = bert_ner(text)
print(ner_results)

[]


## 6. Generación de Texto

### 6.1 **Pregunta 12**: ¿Cómo funcionan los modelos de generación de texto como GPT-2 o GPT-3?

Los modelos de GPT2 y GPT3 son modelos de lenguaje basados en Transformers. Básicamente se usan para la predicción de la siguiente palabra a través de una alimentación de bases de información para poder brindar un contexto acorde a lo que se está solicitando.

El proceso de preentrenamiento de GPT-2 y GPT-3 consiste en exponer al modelo a grandes cantidades de texto de internet (corpus masivos) y hacer que aprenda a predecir la siguiente palabra en una secuencia. Este proceso se llama modelado de lenguaje causal y se basa en el principio de aprendizaje no supervisado.

GPT-2 tiene 1.5 mil millones de parámetros, mientras que GPT-3 es significativamente más grande, con 175 mil millones de parámetros. Esto le da a GPT-3 una capacidad mucho mayor para aprender patrones más complejos y manejar tareas más diversas. Algunas diferencias clave son:

•   Mayor capacidad de comprensión y generación: GPT-3 puede generar texto más coherente y manejar mejor el contexto a largo plazo en comparación con GPT-2.

•   Capacidades multitarea: GPT-3, gracias a su tamaño, es capaz de realizar una variedad de tareas sin necesitar entrenamiento adicional (fine-tuning). Puede traducir texto, responder preguntas, realizar razonamientos matemáticos, y más, simplemente a través de una buena promoción o instrucción (few-shot learning).

•   Few-shot, one-shot, y zero-shot learning: GPT-3 es muy efectivo en estos modos de aprendizaje, donde puede resolver tareas con solo un pequeño número de ejemplos o incluso sin ejemplos.


#### 6.1.1 **Ejercicio**: Usar Hugging Face para generar texto a partir de un prompt con GPT-2.

In [229]:
# Carga del modelo localmente
generator = pipeline('text-generation', model='gpt2')

In [230]:
# Generar texto
response = generator("describe the benefits to eat salads everyday",
                     max_length=1024,
                     do_sample=True,
                     temperature=0.7)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [231]:
print(response[0]['generated_text'])

describe the benefits to eat salads everyday.

How can I get started with a nutrition plan?

To find out more about nutrition, visit the Healthy Hunger-Free Eating Guide.

Find out about any food-related health problems that you might be experiencing.

Find out how you can get more out of your diet.

And for more nutrition tips, visit the Healthy Hunger-Free Living Guide.

Related

Share this: Email

Facebook

LinkedIn

Pinterest

Twitter

Google

Tumblr

Reddit

More

Print

Pocket


Like this: Like Loading...


### 6.2 **Pregunta 13**: ¿Cuáles son los desafíos éticos asociados con los modelos de generación de texto?

Los modelos de generación de texto (NLP), han avanzado significativamente en los últimos años, pero también plantean desafíos éticos cruciales; algunos de los más relevantes pueden ser:
- **Sesgos**: Los modelos aprenden de los datos con los que son entrenados. Si estos datos contienen sesgos sociales, culturales o de otro tipo, el modelo los reproducirá y amplificará en sus salidas. Esto puede llevar a la discriminación, la perpetuación de estereotipos y la generación de contenido ofensivo o dañino.
- **Desinformación y "deepfakes" de texto**: La capacidad de generar texto realista y coherente puede ser utilizada para crear noticias falsas, propaganda o contenido engañoso. Esto plantea serias amenazas a la democracia y la confianza en la información.
- **Privacidad**: Los modelos pueden ser entrenados con grandes cantidades de datos que incluyen información personal. Esto plantea preocupaciones sobre la privacidad y la seguridad de los datos.
- **Responsabilidad**: ¿Quién es responsable del contenido generado por un modelo? ¿El desarrollador, el usuario o el modelo mismo? Esta pregunta es compleja y aún no tiene una respuesta clara.
- **Mal uso**: Los modelos de lenguaje pueden ser utilizados para cometer delitos, como el acoso en línea o la extorsión.


#### 6.2.1 **Ejercicio**: Discutir sobre sesgos en los modelos de lenguaje generativo y su impacto en aplicaciones reales.

 Los sesgos en los modelos de lenguaje generativo tienen un impacto significativo en diversas aplicaciones reales. Algunos ejemplos incluyen:

- **Traducción automática**: Un modelo entrenado con datos sesgados puede generar traducciones que refuerzan estereotipos o discriminan a ciertos grupos.

- **Chatbots y asistentes virtuales**: Un chatbot sesgado puede proporcionar respuestas discriminatorias u ofensivas a preguntas o solicitudes.

- **Generación de contenido creativo**: Un modelo puede generar contenido que perpetúe estereotipos de género, raza u orientación sexual.

- **Toma de decisiones**: Los modelos de lenguaje se utilizan cada vez más para tomar decisiones en áreas como la contratación, la justicia penal y la atención médica. Si estos modelos están sesgados, pueden llevar a resultados injustos.

Para mitigar estos problemas, es necesario:

- **Datos de entrenamiento diversos y representativos**: Los conjuntos de datos utilizados para entrenar los modelos deben ser lo más diversos y representativos posible para reducir la probabilidad de sesgos.

- **Técnicas de mitigación de sesgos**: Existen diversas técnicas para identificar y mitigar los sesgos en los modelos, como la detección de sesgos, la corrección de sesgos y la auditoría de modelos.

- **Transparencia y explicabilidad**: Es importante que los desarrolladores de modelos sean transparentes sobre los datos utilizados para entrenar los modelos y sobre las técnicas de mitigación de sesgos empleadas.

- **Evaluación continua**: Los modelos deben ser evaluados continuamente para detectar y corregir nuevos sesgos que puedan surgir.

## 7. Evaluación de Modelos NLP

### 7.1 **Pregunta 14**: ¿Qué métricas se utilizan para evaluar la calidad de los modelos de NLP en tareas como clasificación o generación de texto?

Existen diversas métricas para evaluar la calidad de los modelos de NLP, dependiendo de la tarea específica. Algunas de las más comunes son:

- **Precisión (Precision), Recuerdo (Recall) y F1-score**: Estas métricas se utilizan comúnmente en tareas de clasificación, como el análisis de sentimientos o la clasificación de textos. Miden la proporción de predicciones correctas, la proporción de elementos relevantes recuperados y el equilibrio entre ambas, respectivamente.
- **BLEU (Bilingual Evaluation Understudy)**: Se utiliza para evaluar la calidad de la traducción automática y la generación de texto. Compara las n-gramas de las traducciones generadas con las de las traducciones de referencia.
- **ROUGE (Recall-Oriented Understudy for Gisting Evaluation)**: Similar a BLEU, pero se enfoca en la recuperación de n-gramas, lo que lo hace más adecuado para tareas de resumen automático.
- **Perplejidad**: Mide la sorpresa de un modelo ante una nueva secuencia de palabras. Un modelo bien entrenado debería tener una baja perplejidad.
- **Coherencia y fluidez**: Para evaluar la calidad de la generación de texto, se pueden utilizar métricas subjetivas como la coherencia y la fluidez del texto generado.

La elección de la métrica adecuada depende aspectos como:
- La tarea específica (clasificación, generación, etc.)
- Los datos disponibles
- Los objetivos del proyecto

En general, el enmascaramiento de palabras es una técnica clave para preentrenar modelos de lenguaje, NER es una tarea fundamental para extraer información de texto, y existen diversas métricas para evaluar la calidad de los modelos de NLP.

#### 7.1.1 **Ejercicio**: Usar métricas como accuracy, precision, y recall para evaluar un modelo de clasificación de texto.

##### 7.1.1.1 Guardar las métricas de accuracy, precision y recall en una lista

In [220]:
metrics = ["accuracy", "recall", "precision", "f1"]

##### 7.1.1.2 Cargar los datos de imdb

In [221]:
data = load_dataset("imdb", split="test").shuffle(seed=42).select(range(1000))

##### 7.1.1.3 Cargar el evalvador para clasificación de textos.

In [222]:
task_evaluator = evaluator("text-classification")

##### 7.1.1.4 Cargar el modelo de clasificación de textos.

In [223]:
imdb_classification = pipeline("text-classification", model="lvwerra/distilbert-imdb")

##### 7.1.1.5 Evaluar los resultados

In [224]:
eval_results = task_evaluator.compute(
    model_or_pipeline = imdb_classification,
    data = data,
    metric = evaluate.combine(metrics),
    label_mapping = {"NEGATIVE": 0, "POSITIVE": 1}
)
print(json.dumps(eval_results, indent=4))

{
    "accuracy": 0.918,
    "recall": 0.9180327868852459,
    "precision": 0.9142857142857143,
    "f1": 0.9161554192229039,
    "total_time_in_seconds": 100.83727800002089,
    "samples_per_second": 9.91696741357688,
    "latency_in_seconds": 0.10083727800002089
}


### 7.2 **Pregunta 15**: ¿Qué es la perplejidad y cómo se utiliza para evaluar modelos de generación de texto?

#### **¿Qué es la perplejidad?**
La perplejidad es uno de las métricas mas comunes para evaluar modelos de lenguaje. Esta métrica aplica específicamente al clásico modelo de lenguaje (a veces llamado autorregresivo o modelos de lenguaje causal) y no es definido como modelos similares a BERT. 

#### **¿Cómo se utiliza para evaluar modelos de generación de texto?**
Se evalúa por factorización autorregresiva una secuencia y condicionando todo el subsecuente precedente en cada paso. Cuando se trabaja con modelos de aproximación, sin embargo, típicamente se tiene una restricción en el numero de tokens del modelo puede proceder. La versión mas grande de GPT-2, por ejemplo, se ha arreglado la longitud de 1024 tokens, entonces no se puede calcular la probabilidad si es mas grande que 1024 tokens.

En cambio, la secuencia esta típicamente dividida en subsecuencias iguales al máximo numero de entradas del modelo. Si el tamaño máximo de la entrada es K, se aproxima la probabilidad condicionando solo K. El token que lo precede en el lugar del texto completo. Cuando se evalúa un modelo de perplejidad de una secuencia, un enfoque puede ser la división en fragmentos disjuntos y sumar las probabilidades logarítmicas descompuestas de cada segmento en forma independiente. 

Esta es la forma rápida para computar desde la perplejidad de cada segmento puede ser computado una pasada hacia adelante, pero funciona como una aproximación pobre de la perplejidad completamente factorizada, por lo cual se producirá un PPL más porque el modelo tendrá menos contexto o genera respuestas erróneas en el resultado de predicción. 


#### 7.2.1 **Ejercicio**: Calcular la perplejidad de un modelo de lenguaje en Hugging Face para una tarea de generación de texto.

##### 7.2.1.1 Utilizar gpt2

In [225]:
device = "cpu"
model_id = "openai-community/gpt2"
model = GPT2LMHeadModel.from_pretrained(model_id).to(device)
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)

c:\Users\guill\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


##### 7.2.1.2 Cargar los datos de WikiText-2

In [226]:
test = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
encodings = tokenizer("\n\n".join(test["text"]), return_tensors="pt")

Token indices sequence length is longer than the specified maximum sequence length for this model (287644 > 1024). Running this sequence through the model will result in indexing errors


##### 7.2.1.3 Calcular la perplejidad

In [227]:
max_length = model.config.n_positions
stride = 512
seq_len = encodings.input_ids.size(1)

nlls = []
prev_end_loc = 0
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # puede ser diferente del paso en el último bucle
    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)

        # la pérdida se calcula utilizando CrossEntropyLoss, que promedia las etiquetas válidas
        # N.B. el modelo solo calcula la pérdida sobre trg_len - 1 etiquetas, porque desplaza las etiqueta internamente
        # a la izquierda por 1.
        neg_log_likelihood = outputs.loss

    nlls.append(neg_log_likelihood)

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break

ppl = torch.exp(torch.stack(nlls).mean())

100%|█████████▉| 560/562 [11:14<00:02,  1.20s/it]


In [228]:
print(f"Perplexity: {ppl.item()}")

Perplexity: 25.187597274780273
